# Initialize

## Import libraries

In [ ]:
import os
import sys
import re
import json
import math
import time
import copy
import io
import random
import datetime
import threading
import signal
import traceback
import cProfile

In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

In [ ]:
import boto3
import panoramasdk

## Configure GPU

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
gpus

In [ ]:
# Don't allocate huge memory unnecessarily
tf.config.experimental.set_memory_growth( gpus[0], True)

## Instantiate panoramasdk.node object

In [ ]:
node = panoramasdk.node()

# Hello World

## Get frames from camera and Put to HDMI

In [ ]:
media_list = node.inputs.video_in.get()

media_list

In [ ]:
media_list[0].image.shape, media_list[0].image.dtype

In [ ]:
def previewImage( image ):
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.figure( figsize = ( 10, 10 ) )
    plt.imshow( image_rgb, interpolation='antialiased' )

In [ ]:
previewImage(media_list[0].image)

In [ ]:
node.outputs.video_out.put( media_list )

## Render "Hello World" text by OpenCV

In [ ]:
text_color = (255,255,255)
text_shadow_color = (0,0,0)
text_thickness = 3
text_shadow_thickness = 5
text_scale = 4

def renderHelloWorld( media_list ):

    image = media_list[0].image
    
    h, w, _ = image.shape
    
    cv2.putText( image, f"Hello World", (22, h//2+2), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=text_scale, color=text_shadow_color, thickness=text_shadow_thickness, lineType=cv2.LINE_AA )
    cv2.putText( image, f"Hello World", (20, h//2), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=text_scale, color=text_color, thickness=text_thickness, lineType=cv2.LINE_AA )


In [ ]:
renderHelloWorld( media_list )

In [ ]:
previewImage(media_list[0].image)

## Main loop (for Hello World)

In [ ]:
def mainLoop():
    try:
        while True:
            
            # get video frame(s) from camera(s)
            media_list = node.inputs.video_in.get()
            
            # render Hello World Text in the image(s)
            renderHelloWorld(media_list)
            
            # output to HDMI
            node.outputs.video_out.put( media_list )
            
    except KeyboardInterrupt:
        pass

In [ ]:
mainLoop()

In [ ]:
!free

# Pose estimation

## Load model

**Manual steps:**

1. Prepare a model file.
    * Option 1 : Download "movenet_multipose_lightning_1.tar.gz" from https://tfhub.dev/google/movenet/multipose/lightning/1 on your PC.
    * Option 2 : Using TensorRT, quantize the movenet_multipose_lightning_1 model, and prepare movenet_multipose_lightning_1_trt_fp16.tar.gz.
2. Upload the prepared model file onto Jupyter (You can Drag & drop the file to browser pane ).
3. Open a Terminal on Jupyter and run following commands.

    ``` sh
    tar xvzf ../movenet_multipose_lightning_1.tar.gz
    ```

In [ ]:
movenet_model = tf.saved_model.load("./movenet_multipose_lightning_1")
#movenet_model = tf.saved_model.load("./movenet_multipose_lightning_1_trt_fp16")

movenet_model

In [ ]:
pose_estimator = movenet_model.signatures["serving_default"]
pose_estimator

In [ ]:
!free

## Preprocess image and run pose estimation

In [ ]:
def estimatePose( image ):

    input_resolution = ( 160, 256 )
    
    image = tf.expand_dims( image, axis=0 )

    image = tf.image.resize( image, input_resolution )

    # BGR to RGB
    image = tf.reverse(image, axis=[-1])

    image = tf.cast( image, dtype=tf.int32 )

    result = pose_estimator(image)

    boxes = result["output_0"].numpy()[:,:,3*17:].reshape(6,5)
    joints = result["output_0"].numpy()[:,:,:3*17].reshape(6,17,3)
    
    return boxes, joints
    

In [ ]:
boxes, joints = estimatePose( media_list[0].image )

boxes, joints

In [ ]:
!free

## Render inference result with OpenCV

In [ ]:
box_threshold = 0.1
joint_threshold = 0.5
box_color = (255,0,0)
box_thickness = 2
dot_color = (0,255,0)
dot_size = 3

def renderResult( image, boxes, joints_list ):
    
    h, w, _ = image.shape
    
    for box, joints in zip( boxes, joints_list ):

        if box[4] < box_threshold:
            continue
        
        box_in_camera_space = (
            int( box[1].item() * w ),
            int( box[0].item() * h ),
            int( box[3].item() * w ),
            int( box[2].item() * h ), 
        )

        cv2.rectangle( 
            image, 
            box_in_camera_space[0:2], 
            box_in_camera_space[2:4], 
            color = box_color, thickness = box_thickness, lineType=cv2.LINE_8
        )
        
        for joint in joints:
            if joint[2] < joint_threshold:
                continue
            
            joint_in_camera_space = (
                int( joint[1].item() * w ) - dot_size,
                int( joint[0].item() * h ) - dot_size,
                int( joint[1].item() * w ) + dot_size,
                int( joint[0].item() * h ) + dot_size, 
            )

            cv2.rectangle( 
                image, 
                joint_in_camera_space[0:2], 
                joint_in_camera_space[2:4], 
                color = dot_color, thickness = -1
            )


In [ ]:
renderResult( media_list[0].image, boxes, joints )

In [ ]:
previewImage(media_list[0].image)

## Main loop (for pose estimation)

In [ ]:
def mainLoop():
    try:
        while True:
            media_list = node.inputs.video_in.get()
            
            for media_obj in media_list:
                boxes, joints = estimatePose( media_obj.image )
                renderResult( media_obj.image, boxes, joints )
            
            node.outputs.video_out.put( media_list )
            
    except KeyboardInterrupt:
        pass


In [ ]:
mainLoop()

In [ ]:
!free

# Tips

## Run the main loop with profiler

In [ ]:
cProfile.runctx( "mainLoop()", globals(), locals() )